In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
train=pd.read_csv('../input/digit-recognizer/train.csv')
test=pd.read_csv('../input/digit-recognizer/test.csv')

x_train=train.drop('label',axis=1)
x_train=np.array(x_train)/255.
x_train=x_train.reshape(-1,28,28,1)
x_train=tf.image.resize(x_train,size=(32,32))
y_train=train['label']

x_test=test
x_test=np.array(x_test)/255.
x_test=x_test.reshape(-1,28,28,1)
x_test=tf.image.resize(x_test,size=(32,32))

In [ ]:
vgg=VGG16(include_top=False,weights='imagenet')

#connect input
input=tf.keras.Input(shape=(32,32,1))
x=tf.keras.layers.Conv2D(3,1)(input)
x=vgg(x)

#connect output
x=tf.keras.layers.Flatten()(x)
x=tf.keras.layers.Dense(120,activation='relu')(x)
x=tf.keras.layers.Dense(84,activation='relu')(x)
output=tf.keras.layers.Dense(10,activation='softmax')(x)

model=tf.keras.Model(input,output)

In [ ]:
model.summary()

In [ ]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='sgd')
model.fit(x_train,y_train,epochs=10)

In [ ]:
y_test=model.predict(x_test)
result=np.argmax(y_test,axis=1)
result_CSV=pd.DataFrame({'ImageId':range(1,len(result)+1),'Label':result})
result_CSV=result_CSV.set_index('ImageId')
result_CSV.to_csv('submission.csv')